In [ ]:
!pip install scipy
!pip install geopy

In [5]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
import csv
from geopy.distance import great_circle # type: ignore


In [26]:
def read_coordinates_with_id_from_csv(filename, id_column, encoding='utf-8'):
    df = pd.read_csv(filename, delimiter=';', encoding=encoding)
    if id_column not in df.columns:
        raise KeyError(f"Column '{id_column}' not found in CSV file.")
    if 'LAT' not in df.columns or 'LON' not in df.columns:
        raise KeyError("Columns 'LAT' and 'LON' must be present in CSV file.")
    
    # Substitui vírgulas por pontos e converte para float, removendo linhas com dados inválidos
    df['LAT'] = df['LAT'].str.replace(',', '.').astype(float)
    df['LON'] = df['LON'].str.replace(',', '.').astype(float)
    
    # Remove linhas com valores NaN ou infinitos
    df = df.dropna(subset=['LAT', 'LON'])
    df = df[np.isfinite(df['LAT']) & np.isfinite(df['LON'])]

    coordinates = df[['LAT', 'LON']].to_numpy()
    ids = df[id_column].tolist()
    
    return coordinates, ids

In [27]:
def calculate_distance(point1, point2):
    return great_circle(point1, point2).meters

def find_closest_postes(sgo_coord, rede_metro_coords, rede_metro_ids):
    distances = []
    valid_coords = []
    
    for coord in rede_metro_coords:
        if np.isfinite(coord).all():  # Verifica se todas as coordenadas são finitas
            distances.append(calculate_distance(sgo_coord, coord))
            valid_coords.append(coord)
    
    distances = np.array(distances)
    close_postes_indices = np.where(distances <= 200)[0]
    close_postes = [(distances[i], rede_metro_ids[i]) for i in close_postes_indices]
    close_postes.sort()
    return close_postes



In [28]:
# Leitura dos arquivos CSV e captura de erros
try:
    rede_metro_coords, rede_metro_ids = read_coordinates_with_id_from_csv('data1.csv', id_column='BARRAMENTO', encoding='latin-1')
except KeyError as e:
    print(f"Erro ao ler dados da REDE_METRO: {e}")
except UnicodeDecodeError:
    print("Erro de decodificação Unicode ao ler o arquivo CSV. Verifique o encoding.")

try:
    sgo_coords, sgo_ids = read_coordinates_with_id_from_csv('data_sgo.csv', id_column='NOTA', encoding='latin-1')
except KeyError as e:
    print(f"Erro ao ler dados da SGO: {e}")
except UnicodeDecodeError:
    print("Erro de decodificação Unicode ao ler o arquivo CSV. Verifique o encoding.")

In [29]:
# Processamento dos dados e cálculo dos postes mais próximos
if 'rede_metro_coords' in locals() and 'sgo_coords' in locals():
    results = []
    for sgo_coord, sgo_id in zip(sgo_coords, sgo_ids):
        if np.isfinite(sgo_coord).all():  # Verifica se as coordenadas SGO são finitas
            close_postes = find_closest_postes(sgo_coord, rede_metro_coords, rede_metro_ids)
            if close_postes:
                distance, closest_poste_id = close_postes[0]
                if distance <= 250:
                    results.append(f"Ponto SGO {sgo_id} - Poste mais próximo {closest_poste_id} - Distância: {distance:.2f} metros")
    
    # Imprime os resultados de forma rápida
    print("\n".join(results))

Ponto SGO 1009610667 - Poste mais próximo 893037 - Distância: 15.32 metros
Ponto SGO 1009574967 - Poste mais próximo 3091195 - Distância: 17.64 metros
Ponto SGO 1009512580 - Poste mais próximo 909325 - Distância: 86.26 metros
Ponto SGO 1009447299 - Poste mais próximo C1031520 - Distância: 13.52 metros
Ponto SGO 1009438814 - Poste mais próximo 896302 - Distância: 104.74 metros
Ponto SGO 1009437963 - Poste mais próximo 637950 - Distância: 185.58 metros
Ponto SGO 1009419952 - Poste mais próximo 23038889 - Distância: 121.50 metros
Ponto SGO 1009404303 - Poste mais próximo C256073 - Distância: 10.82 metros
Ponto SGO 1009390289 - Poste mais próximo 1032378 - Distância: 95.27 metros
Ponto SGO 1009390205 - Poste mais próximo 651827 - Distância: 17.70 metros
Ponto SGO 1009389395 - Poste mais próximo 00299471 - Distância: 1.27 metros
Ponto SGO 1009387271 - Poste mais próximo IP04090694 - Distância: 62.98 metros
Ponto SGO 1009374670 - Poste mais próximo 969753 - Distância: 44.50 metros
Ponto SGO 